In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
from tqdm import tqdm

In [2]:
%%html
<style>
.container { width:100% !important; }
</style>

```
 医院概况
    √华山之路    标题、内容
    ×现任领导    几乎无内容
    ×历任领导    几乎无内容
 新闻动态
     √图文新闻   列表        标题、内容（图文）
     √华山公告   列表        标题、内容（图文）
     √媒体视点   列表        微信公众号
     √视频宣传   列表        无内容
 门诊导航
     √预约挂号   标题内容
     ×就医流程   图
     √楼层分布   标题、内容（中英混杂）
     ×来院交通   图文
     ×出入院须知  图
 学科介绍
     √外科系统   部门列表     人员列表、子标题、内容
     √内科系统   部门列表     人员列表、子标题、内容
     √医技科室   部门列表     人员列表、子标题、内容
     √行政部门   部门列表     人员列表、子标题、内容
 专家介绍
     √院士风采   列表        标题、内容
     √终身教授   列表        标题、内容
 科研教育
     ×教育培训       另一网站
     ×伦理委员会      列表        无内容
     √药物临床试验机构  列表        资源下载、标题内容
 人才招聘
     √招聘公告       列表        标题、内容
     ×投递简历       列表        无文本内容
 党建园地
     ×无法访问
 院务公开
     ×执业信息       列表        无内容
     √捐赠公示       列表        标题、内容（图文）
     √服务公告       列表        标题、内容
     √服务价格及收费   列表        标题、内容
     √预决算信息      列表        标题、内容
     √招投标公告      列表        标题、内容
     ×项目查询       无内容
     ×投诉建议       无内容
 专题网站
     华馨家园志愿者服务    标题内容
     √华馨家园动态        列表        标题、内容
     √培训资料下载        列表        标题、图片
     √志愿者风采         列表        图片
     ×志愿者报名         无内容
     √援外支边公益行       列表        标题、内容
     ×国家老年疾病临床医学研究中心  另外网站
     ×国家传染病医学中心    另外网站
     
```

In [3]:
driver = webdriver.Chrome()

# 传入包含列表的页面
# 获取当前页面中的版块名、子版块名
# 遍历子版块中的内容
news_url = "https://www.huashan.org.cn/news/1000/index.html"
driver.get(news_url)


subtitle = driver.find_element(by=By.XPATH,value='/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[1]/tbody/tr/td/span').text
total_page = int(re.findall("共(.*?)页",driver.find_element(by=By.XPATH,value='/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[14]/tbody/tr/td/div/span[1]').text)[0])


index = 1 
url_list = list()
while index <= total_page:
    content_list = driver.find_elements(by=By.XPATH,value='/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a')
    url_list.extend([content.get_attribute('href') for content in content_list])
    index += 1 
    nextpage_btn = driver.find_element(by=By.XPATH,value='/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[14]/tbody/tr/td/div/a[1]/img')
    nextpage_btn.click()
    time.sleep(2)

In [4]:
def url_list_catching(base_url
                      ,content_list_xpath
#                       ,subtitle_xpath = "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[1]/tbody/tr/td/span"
                      ,total_page_xpath = "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[14]/tbody/tr/td/div/span[1]"
                      #,content_list_xpath = '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a'
                      ,jump_btn_xpath = '//*[@id="go"]'
                     ):
    url_list = list()
    driver.get(base_url)
    
    try:
        total_page = int(re.findall("共(.*?)页",driver.find_element(by=By.XPATH,value=total_page_xpath).text)[0])
    except:
        time.sleep(2)
        content_list = driver.find_elements(by=By.XPATH,value=content_list_xpath)  
        url_list.extend([content.get_attribute('href') for content in content_list])
        return url_list
    index = 1 
    while index <= total_page:
        
        content_list = driver.find_elements(by=By.XPATH,value=content_list_xpath)  
        url_list.extend([content.get_attribute('href') for content in content_list])
        jump_btn = driver.find_element(by=By.XPATH,value=jump_btn_xpath)
        index += 1
        time.sleep(2)
        index_inputs = driver.find_element(by=By.XPATH,value='//*[@id="pshow"]')
        index_inputs.clear()
        index_inputs.send_keys(index)
        jump_btn.click()
    return url_list


def url_list_catching_defeated(base_url
                      ,subtitle_xpath = "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[1]/tbody/tr/td/span"
                      ,total_page_xpath = "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[14]/tbody/tr/td/div/span[1]"
                      ,content_list_xpath = '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a'
                      ,jump_btn_xpath = '//*[@id="go"]'
                     ):
    driver.get(base_url)
    subtitle = driver.find_element(by=By.XPATH,value=subtitle_xpath).text
    total_page = int(re.findall("共(.*?)页",driver.find_element(by=By.XPATH,value=total_page_xpath).text)[0])
    
    index = 1 
    url_list = list()
    while index <= total_page:
        jump_btn.click()
        content_list = driver.find_elements(by=By.XPATH,value=content_list_xpath)  
        url_list.extend([content.get_attribute('href') for content in content_list])
        jump_btn = driver.find_element(by=By.XPATH,value=jump_btn_xpath)
        index += 1
        time.sleep(2)
        index_inputs = driver.find_element(by=By.XPATH,value='//*[@id="pshow"]')
        index_inputs.clear()
        index_inputs.send_keys(index)
    return subtitle,url_list


In [5]:
def huashan_news_catching(url
                          ,context_title_xpath = None
                          ,context_xpath = None
                         ):
    if context_xpath is None:
        context_xpath = "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[6]/tbody/tr/td/p/span"
    if context_title_xpath is None:
        context_title_xpath = '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[2]'
    driver.get(url)
    title = driver.find_element(by=By.XPATH,value=context_title_xpath).text
    context = [_.text for _ in driver.find_elements(by=By.XPATH,value=context_xpath) if len(_.text) > 0 ]
    return title,context
def weixin_info_catching(url
                         ,context_title_xpath = None
                         ,context_xpath = None
                        ):
    if context_xpath is None:
        context_xpath = '//*[@id="js_content"]/section'
    if context_title_xpath is None:
        context_title_xpath = '//*[@id="activity-name"]'
    driver.get(url)
    title = driver.find_element(by=By.XPATH,value=context_title_xpath).text
    _content = driver.find_element(by=By.XPATH,value=context_xpath)
    _content = _content.find_elements(by=By.XPATH,value="./child::*")
    content = [_.text for _ in _content if len(_.text) > 0]
    return title,content

# 华山公告
subtitle,url_list = url_list_catching("https://www.huashan.org.cn/news/1001/index.html")

In [6]:
catching_method = {
    "huashan" : huashan_news_catching,
    "weixin" : weixin_info_catching
}
list_contained_page = [
#     ("https://www.huashan.org.cn/news/1000/index.html",'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a'),#图文新闻
#     ("https://www.huashan.org.cn/news/1001/index.html",'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a'),#华山公告
#     ("https://www.huashan.org.cn/news/1002/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#媒体视点
#     ("https://www.huashan.org.cn/yuanwu/1032/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#招投标公告
#     ("https://www.huashan.org.cn/zhuanti/1009/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#华馨家园动态
#     ("https://www.huashan.org.cn/zhuanti/1181/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#培训资料下载
#     ("https://www.huashan.org.cn/zhuanti/1182/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#志愿者风采
#     ("https://www.huashan.org.cn/zhuanjia/1037/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/a"),#院士风采
#     ("https://www.huashan.org.cn/zhuanjia/1038/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/a"),#终身教授
#     ("https://www.huashan.org.cn/xueke/1033/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a"),#外科系统
#     ("https://www.huashan.org.cn/xueke/1034/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a"),#内科系统
#     ("https://www.huashan.org.cn/xueke/1035/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td/a"),#医技科室
#     ("https://www.huashan.org.cn/xueke/1036/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a"),#行政部门
#     ("https://www.huashan.org.cn/cms/1196.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#药物临床试验机构
#     ("https://www.huashan.org.cn/xueshu/1015/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#招聘公告
#     ("https://www.huashan.org.cn/yuanwu/1013/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#捐赠公示
#     ("https://www.huashan.org.cn/yuanwu/1014/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#服务公告
#     ("https://www.huashan.org.cn/yuanwu/1029/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#服务价格及收费
#     ("https://www.huashan.org.cn/yuanwu/1215/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#预决算信息
#     ("https://www.huashan.org.cn/zhuanti/1005/index.html","/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a"),#援外支边公益行
]
list_contained_page = [
    {
        "part_name" : "图文新闻",
        "url":"https://www.huashan.org.cn/news/1000/index.html",
        "content_list_xpath" : '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
        "context_xpath" : None
    }
    ,{
        "part_name":"华山公告",
        "url":"https://www.huashan.org.cn/news/1001/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
        "context_xpath" : None
    }
    ,{
        "part_name":"媒体视点",
        "url":"https://www.huashan.org.cn/news/1002/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
        "context_xpath" : None
    }
        ,{
        "part_name":"招投标公告",
        "url":"https://www.huashan.org.cn/news/1002/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
            "context_xpath" : None
    }
        ,{
        "part_name":"华馨家园动态",
        "url":"https://www.huashan.org.cn/zhuanti/1009/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
            "context_xpath" : None
    }
        ,{
        "part_name":"培训资料下载",
        "url":"https://www.huashan.org.cn/zhuanti/1181/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
            "context_xpath" : None
    }
        ,{
        "part_name":"志愿者风采",
        "url":"https://www.huashan.org.cn/zhuanti/1182/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
            "context_xpath" : None
    }
        ,{
        "part_name":"院士风采",
        "url":"https://www.huashan.org.cn/zhuanjia/1037/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/a',
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/table/tbody/tr/td[2]/p"

        }
            ,{
        "part_name":"终身教授",
        "url":"https://www.huashan.org.cn/zhuanjia/1038/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/a',
                "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/table/tbody/tr/td[2]/p"
        
            }
        ,{
        "part_name":"外科系统",
        "url":"https://www.huashan.org.cn/xueke/1033/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a",
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/p"
    }
            ,{
        "part_name":"内科系统",
        "url":"https://www.huashan.org.cn/xueke/1034/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a",
                "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/p"
    }
        ,{
        "part_name":"医技科室",
        "url":"https://www.huashan.org.cn/xueke/1035/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td/a',
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/p"
    }
            ,{
        "part_name":"行政部门",
        "url":"https://www.huashan.org.cn/xueke/1036/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td[2]/table/tbody/tr/td/table/tbody/tr/td[2]/a",
                "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[4]/tbody/tr/td/p"
    }
        ,{
        "part_name":"药物临床试验机构",
        "url":"https://www.huashan.org.cn/cms/1196.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
            "context_xpath" : None
    }
            ,{
        "part_name":"招聘公告",
        "url":"https://www.huashan.org.cn/xueshu/1015/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td[2]/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
                "context_xpath" : None
    }
        ,{
        "part_name":"捐赠公示",
        "url":"https://www.huashan.org.cn/yuanwu/1013/index.html",
        "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[6]/tbody/tr/td/p"
    }
            ,{
        "part_name":"服务公告",
        "url":"https://www.huashan.org.cn/yuanwu/1014/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
                "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[6]/tbody/tr/td/font/p"
    }
        ,{
        "part_name":"服务价格及收费",
        "url":"https://www.huashan.org.cn/yuanwu/1029/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table[6]/tbody/tr/td/p"
    }
#             ,{
#         "part_name":"预决算信息",
#         "url":"https://www.huashan.org.cn/yuanwu/1215/index.html",
#         "content_list_xpath":'/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a',
#                 "context_xpath" : None
#     }
        ,{
        "part_name":"援外支边公益行",
        "url":"https://www.huashan.org.cn/zhuanti/1005/index.html",
        "content_list_xpath":"/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table/tbody/tr/td[2]/a",
            "context_xpath" : None
    }
]

In [7]:
total_data = list()
for part in list_contained_page:
    url_list = url_list_catching(part["url"],part["content_list_xpath"])
    for url in tqdm(url_list):
        time.sleep(2)
        try:
            if "huashan" in url.split("."):
                title,content = catching_method["huashan"](url,part["context_xpath"])
            if "weixin" in url.split("."):
                title,content = catching_method["weixin"](url)
            item = {
                "part" : part["part_name"],
                "title":title,
                "content":content
            }
            total_data.append(item)
        except:
            continue

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.91s/it]


In [11]:
import json
import os
path = os.path.join(os.getcwd(),"data")
path = os.path.join(path,"crawler.json")
with open(path,"w",encoding="utf-8") as f:
    json.dump(total_data,f,ensure_ascii=False,indent=2)

In [ ]:
single_page = [
        {
        "part_name":"华山之路",
        "url":"https://www.huashan.org.cn/about/1018/index.html",
            "context_title_xpath" : '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[1]/tbody/tr/td/span',
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/p",
    }
    ,        {
        "part_name":"预约挂号",
        "url":"https://www.huashan.org.cn/menzhen/1017/index.html",
            "context_xpath" : '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/p',
        "context_title_xpath" : '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[1]/tbody/tr/td/img'
    }
        ,        {
        "part_name":"楼层分布",
        "url":"https://www.huashan.org.cn/menzhen/1026/index.html",
            "context_xpath" : "/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[3]/tbody/tr/td/table/tbody/tr/td/table[2]/tbody/tr/td/p",
            "context_title_xpath" : '/html/body/div/table[3]/tbody/tr/td/table/tbody/tr/td/table/tbody/tr/td[3]/table[1]/tbody/tr/td/img'
    }
    
    
]
for single in single_page:
    url = single["url"]
    try:
        if "huashan" in url.split("."):
            title,content = catching_method["huashan"](url,single["context_title_xpath"],single["context_xpath"])
        if "weixin" in url.split("."):
            title,content = catching_method["weixin"](url,single["context_title_xpath"],context_xpath=single["context_xpath"])
        item = {
            "part" : single["part_name"],
            "title":title,
            "content":content
        }
        total_data.append(item)
    except:
        continue